In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import mean_squared_error, r2_score
pd.options.mode.chained_assignment = None 
from matplotlib import pyplot
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPRegressor
import pickle
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import cross_validate, cross_val_score
import csv
from sklearn.preprocessing import StandardScaler
pd.set_option('display.max_columns', None)

In [2]:
def Final_Clean(df, direction):
  
    df = df.loc[df['DIRECTION'] == direction]
    
    #covert to appropriate data types
    df['DAYOFSERVICE'] = df['DAYOFSERVICE'].astype('datetime64[ns]')
    df['ROUTEID'] = df['ROUTEID'].astype('category')
    df['LINEID'] = df['LINEID'].astype('category')

    #adding an hour column so that dataframe can be merged with weather dataframe 
    df['HOUR'] = pd.to_datetime(df['DEPARTURE_TIME_ACTUAL'])
    df['HOUR'] = df['HOUR'].dt.hour

    #Grouping the trips by day of service and trip ID and then 
    #sorting the dataframe according to progrnumber (Sequential position of the stop point in the trip)
    sorted_df = df.groupby(["DAYOFSERVICE","TRIPID"]).apply(lambda x: x.sort_values(["PROGRNUMBER"], ascending = True)).reset_index(drop=True)

    #Creating a feature for the duration of the journey 
    #this is done by taking the actual arrival time of the preceding row from the arrival time of the following row.
    sorted_df['JOURNEY_DURATION'] = sorted_df['ACTUALTIME_ARR'].shift(-1) - sorted_df['ACTUALTIME_ARR'] - sorted_df['DWELL_TIME']

    #creating the peak hour feature
    sorted_df['PEAK_HOUR'] = [1 if x >'07:00:00' and x <'10:00:00' or x >'16:00:00' and x <'19:00:00' else 0 for x in sorted_df['ARRIVAL_TIME_ACTUAL']]

    #creating a public holiday indicator feature 
    dates = pd.to_datetime(['2018-01-01','2018-03-17','2018-03-19','2018-03-30','2018-04-01','2018-04-02','2018-05-07',
     '2018-06-04','2018-08-06','2018-11-29','2018-12-24','2018-12-25','2018-12-26', '2018-12-31'])

    sorted_df['PUBLIC_HOLIDAY'] = sorted_df['DAYOFSERVICE'].isin(dates)
    sorted_df['PUBLIC_HOLIDAY'] *= 1    
    
    #deleting columns that won't be used
    del sorted_df['ARRIVAL_TIME_ACTUAL']
    del sorted_df['DEPARTURE_TIME_ACTUAL']
    del sorted_df['PLANNEDTIME_DEP']
    del sorted_df['PLANNEDTIME_ARR']

    #we want to get rid of all rows where the bus has reached the final destination of the trip
    #the journey duration of these rows is not representative of the actual journey time as 
    #this is the time spent in the bay before the next trip 

    #we create a column END_OF_TRIP that will be 1 for all in transit journeys
    #and negative for the final destination as it is the subtraction of the progress number of the current section
    #from the progress number of the next section of the trip 
    sorted_df['NEXT_STOPPOINTID'] = sorted_df['STOPPOINTID'].shift(-1)
    sorted_df['NEXT_PROGNUMBER']=sorted_df["PROGRNUMBER"].shift(-1)
    sorted_df['END_OF_TRIP'] = sorted_df['NEXT_PROGNUMBER'] - sorted_df['PROGRNUMBER']

    #as the final rows journey duration, NEXT_STOPPOINTID, NEXT_PROGNUMBER and END_OF_TRIP are null
    sorted_df = sorted_df.dropna(axis=0)

    
    #merging weather data 
    weather_df = pd.read_csv('~/data/Clean_Weather_2018.csv')
    weather_df['date'] = weather_df['date'].astype('datetime64[ns]')
    weather_df['weather_main'] = weather_df['weather_main'].astype('category')

    final_df = pd.merge(sorted_df, weather_df, how='left',left_on=['DAYOFSERVICE','HOUR'],right_on=['date','hour'])

    
    del final_df['VEHICLEID']
    del final_df['date']
    del final_df['hour']

    final_df['MONTH']= final_df['MONTH'].astype('category')
    final_df['DAY']= final_df['DAY'].astype('category')
    final_df['HOUR']= final_df['HOUR'].astype('category')
    final_df['STOPPOINTID']= final_df['STOPPOINTID'].astype('category')
    final_df['NEXT_STOPPOINTID']= final_df['NEXT_STOPPOINTID'].astype('int')
    final_df['NEXT_STOPPOINTID']= final_df['NEXT_STOPPOINTID'].astype('category')
    final_df['JOURNEY_DURATION']= final_df['JOURNEY_DURATION'].astype('int')

    #removing the row at the end of each trip where journey duration is not required.
    final_df = final_df.loc[final_df['END_OF_TRIP'] == 1]
    
    #remove negative journey times
    final_df = final_df[(final_df['JOURNEY_DURATION'] > 0)]
    
    #remove outliers that are 3 standard deviations above the mean.
    final_df=final_df[np.abs(final_df.JOURNEY_DURATION-final_df.JOURNEY_DURATION.mean()) <= (3*final_df.JOURNEY_DURATION.std())]

    return final_df

In [3]:
def Lin_reg(df):

    X = pd.DataFrame(df[['DAY','HOUR','MONTH','STOPPOINTID','NEXT_STOPPOINTID',
                           'PEAK_HOUR', 'wind_speed', 'temp', 'humidity', 'weather_main']])
    y = df.JOURNEY_DURATION
    X = pd.get_dummies(X)
    
    dummies = X.columns
    
    with open("/home/tomah/data/Model_Testing/LinRegPickles_2/LinReg_route__{}_d{}_headers.pkl".format(Route, direction), 'wb') as handle:
        pickle.dump(dummies, handle, pickle.HIGHEST_PROTOCOL)

    #test-train split 
    X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.3, random_state=1)

    #creating and fitting the linear model 
    regression = linear_model.LinearRegression()
    regression.fit(X_train,Y_train)
    
    with open("/home/tomah/data/Model_Testing/LinRegPickles_2/LinReg_Model_route_{}_d{}.pkl".format(Route, direction), 'wb') as handle:
        pickle.dump(regression, handle, pickle.HIGHEST_PROTOCOL)

    #predictions from the training set
    y_pred_linear_train = regression.predict(X_train)
    
    #predicitng from the test set 
    y_pred_linear = regression.predict(X_test)
    
    #some evaluation metrics.
    print('Mean Absolute Error (train):', metrics.mean_absolute_error(Y_train, y_pred_linear_train))
    print('Mean Absolute Error (test):', metrics.mean_absolute_error(Y_test, y_pred_linear))
    print('Mean Percentage Error:', np.mean((Y_test - y_pred_linear)/Y_test))
    print('Mean Absolute Percentage Error:', metrics.mean_absolute_percentage_error(Y_test,y_pred_linear))    
    print('Mean Squared Error:', metrics.mean_squared_error(Y_test, y_pred_linear))
    print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(Y_test, y_pred_linear)))
    print('R-square value:', r2_score(Y_test,y_pred_linear))
    
    Mean_abs_error_train = metrics.mean_absolute_error(Y_train, y_pred_linear_train)
    Mean_abs_error_test = metrics.mean_absolute_error(Y_test, y_pred_linear)
    Mean_Percentage_Error = np.mean((Y_test - y_pred_linear)/Y_test)
    Mean_Absolute_Percentage_Error= metrics.mean_absolute_percentage_error(Y_test,y_pred_linear)  
    Mean_squared_error =  metrics.mean_squared_error(Y_test, y_pred_linear)
    Root_Mean_sqaured_error = np.sqrt(metrics.mean_squared_error(Y_test, y_pred_linear))
    R_square_value =  r2_score(Y_test,y_pred_linear)
    
    with open('Linear_Regression_Results_direction_split.csv', 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([Route, direction, Mean_abs_error_train, Mean_abs_error_test,Mean_Percentage_Error,
                         Mean_Absolute_Percentage_Error, Mean_squared_error, Root_Mean_sqaured_error, R_square_value])

In [4]:
Trips_df = pd.read_csv("~/data/rt_trips_DB_2018.txt", ";")
Line_List = Trips_df.LINEID.unique().tolist()

# By Route

In [29]:
with open('Linear_Regression_Results.csv', 'a', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Route","Mean_Absolute_Error_Train","Mean_Absolute_Error_Test", 
                     "Mean_Percentage_Error", "Mean_Absolute_Percentage_Error",
                     "Mean_Squared_Error", "Root_Mean_Squared_Error", "R_Square"])

for Route in Line_List:
    df = pd.read_csv("~/data/Cleaned_By_Route_Header/route_{}.csv".format(Route))
    print("//--------------------------------------//")
    print("route:",Route,"\n")
    final_df = Final_Clean(df)
    Lin_reg(final_df)   

//--------------------------------------//
route: 68 

Mean Absolute Error (train): 11.437559214802521
Mean Absolute Error (test): 11.4332248502619
Mean Percentage Error: -0.10364038697514714
Mean Absolute Percentage Error: 0.2811026425965462
Mean Squared Error: 354.67756392849566
Root Mean Squared Error: 18.832885172710412
R-square value: 0.6982585602951237
//--------------------------------------//
route: 25B 

Mean Absolute Error (train): 15.87242928352212
Mean Absolute Error (test): 15.90120894738207
Mean Percentage Error: -0.09877668234611595
Mean Absolute Percentage Error: 0.28142327775312015
Mean Squared Error: 602.3632468967827
Root Mean Squared Error: 24.54308959558235
R-square value: 0.7226799074517014
//--------------------------------------//
route: 45A 

Mean Absolute Error (train): 10.844545557017945
Mean Absolute Error (test): 10.85706625665745
Mean Percentage Error: -0.08409290858770209
Mean Absolute Percentage Error: 0.2500354407670376
Mean Squared Error: 279.816330766

//--------------------------------------//
route: 59 

Mean Absolute Error (train): 7.952803794399173
Mean Absolute Error (test): 7.925666784250906
Mean Percentage Error: -0.06473645265241167
Mean Absolute Percentage Error: 0.19982983857877562
Mean Squared Error: 149.75281605294535
Root Mean Squared Error: 12.23735331078356
R-square value: 0.7147785535786082
//--------------------------------------//
route: 9 

Mean Absolute Error (train): 14.695218835989067
Mean Absolute Error (test): 14.714935221542612
Mean Percentage Error: -0.10975307462097456
Mean Absolute Percentage Error: 0.2904900691241592
Mean Squared Error: 497.517629399453
Root Mean Squared Error: 22.305103214274823
R-square value: 0.64313668314642
//--------------------------------------//
route: 40 

Mean Absolute Error (train): 13.489016810481107
Mean Absolute Error (test): 13.478374210499505
Mean Percentage Error: -0.094034104775824
Mean Absolute Percentage Error: 0.2683616195619331
Mean Squared Error: 483.2246522298402


//--------------------------------------//
route: 184 

Mean Absolute Error (train): 9.06052496852208
Mean Absolute Error (test): 9.075619359476676
Mean Percentage Error: -0.05759453663166792
Mean Absolute Percentage Error: 0.18570245415279318
Mean Squared Error: 241.1716076027674
Root Mean Squared Error: 15.529700821418531
R-square value: 0.8389187255148655
//--------------------------------------//
route: 238 

Mean Absolute Error (train): 10.6518624010764
Mean Absolute Error (test): 10.6729517780877
Mean Percentage Error: -0.059597018802094136
Mean Absolute Percentage Error: 0.18830325541879456
Mean Squared Error: 397.3427795287363
Root Mean Squared Error: 19.933458794919066
R-square value: 0.781443074646847
//--------------------------------------//
route: 145 

Mean Absolute Error (train): 15.601309287340015
Mean Absolute Error (test): 15.588191739568156
Mean Percentage Error: -0.12081492385136114
Mean Absolute Percentage Error: 0.3106437680436173
Mean Squared Error: 577.355566097

//--------------------------------------//
route: 4 

Mean Absolute Error (train): 15.507837270036564
Mean Absolute Error (test): 15.540923608370287
Mean Percentage Error: -0.12487420366373626
Mean Absolute Percentage Error: 0.31994500045120744
Mean Squared Error: 508.83768295264804
Root Mean Squared Error: 22.557430770206256
R-square value: 0.6874421152902734
//--------------------------------------//
route: 120 

Mean Absolute Error (train): 13.175276255145846
Mean Absolute Error (test): 13.149764421265724
Mean Percentage Error: -0.10363654839918301
Mean Absolute Percentage Error: 0.2637393712981372
Mean Squared Error: 442.79453689632
Root Mean Squared Error: 21.042683690449753
R-square value: 0.68712691858715
//--------------------------------------//
route: 41C 

Mean Absolute Error (train): 13.952357468585273
Mean Absolute Error (test): 13.919067640496662
Mean Percentage Error: -0.11877286724177626
Mean Absolute Percentage Error: 0.3174070344049483
Mean Squared Error: 410.43244496

Mean Absolute Error (train): 14.59429933209971
Mean Absolute Error (test): 14.507720439746327
Mean Percentage Error: -0.11292348920098495
Mean Absolute Percentage Error: 0.2718376005480061
Mean Squared Error: 546.118401367949
Root Mean Squared Error: 23.369176309145963
R-square value: 0.5516255874968816
//--------------------------------------//
route: 161 

Mean Absolute Error (train): 15.380532941030495
Mean Absolute Error (test): 15.66335283120186
Mean Percentage Error: -0.14065769281017473
Mean Absolute Percentage Error: 0.31651484628967524
Mean Squared Error: 815.1413210077615
Root Mean Squared Error: 28.55067986944902
R-square value: 0.5550478471535845
//--------------------------------------//
route: 7A 

Mean Absolute Error (train): 13.843597137661199
Mean Absolute Error (test): 13.824594828441617
Mean Percentage Error: -0.11294657117486367
Mean Absolute Percentage Error: 0.30122005736600016
Mean Squared Error: 467.5920297999327
Root Mean Squared Error: 21.623876382368003
R-squ

//--------------------------------------//
route: 66B 

Mean Absolute Error (train): 13.920856699622014
Mean Absolute Error (test): 13.888324899569088
Mean Percentage Error: -0.1033276172421739
Mean Absolute Percentage Error: 0.2787409461022525
Mean Squared Error: 539.4716709873696
Root Mean Squared Error: 23.226529464975382
R-square value: 0.7016257842038109
//--------------------------------------//
route: 38B 

Mean Absolute Error (train): 14.661856532322632
Mean Absolute Error (test): 14.469146312583787
Mean Percentage Error: -0.11035032518043875
Mean Absolute Percentage Error: 0.2835263738210341
Mean Squared Error: 552.8267620114655
Root Mean Squared Error: 23.51226832977766
R-square value: 0.7288120876538593
//--------------------------------------//
route: 236 

Mean Absolute Error (train): 10.27912286824024
Mean Absolute Error (test): 10.756834356570883
Mean Percentage Error: -0.06292379595647095
Mean Absolute Percentage Error: 0.17912388771779297
Mean Squared Error: 415.352717

# By route in each direction

In [17]:
with open('Linear_Regression_Results_direction_split.csv', 'a', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Route","Direction", "Mean_Absolute_Error_Train","Mean_Absolute_Error_Test", 
                     "Mean_Percentage_Error", "Mean_Absolute_Percentage_Error",
                     "Mean_Squared_Error", "Root_Mean_Squared_Error", "R_Square"])

        
for Route in Line_List:

    df = pd.read_csv("~/data/Cleaned_By_Route_Header/route_{}.csv".format(Route))
    print("//--------------------------------------//")  
    
    if len(df.DIRECTION.unique()) == 1:

        direction = df.DIRECTION.unique()[0]
        print("route:",Route,"\n","direction:", direction)
        final_df = Final_Clean(df, direction)
        Lin_reg(final_df)  
    
    elif len(df.DIRECTION.unique()) == 2:
        for direction in range(1,3):
            print("route:",Route,"\n","direction:", direction)
            final_df = Final_Clean(df, direction)
            Lin_reg(final_df)

//--------------------------------------//
route: 68 
 direction: 1
Mean Absolute Error (train): 10.675359558366731
Mean Absolute Error (test): 10.64593203889172
Mean Percentage Error: -0.09763204826932369
Mean Absolute Percentage Error: 0.26652475594920755
Mean Squared Error: 296.8582618868292
Root Mean Squared Error: 17.229575209123098
R-square value: 0.7466076477218522
route: 68 
 direction: 2
Mean Absolute Error (train): 12.085706625514387
Mean Absolute Error (test): 12.056217635762756
Mean Percentage Error: -0.11082051052365012
Mean Absolute Percentage Error: 0.293586017948098
Mean Squared Error: 403.4227634749617
Root Mean Squared Error: 20.085386814173177
R-square value: 0.6432428930386963
//--------------------------------------//
route: 25B 
 direction: 1
Mean Absolute Error (train): 14.686820554596451
Mean Absolute Error (test): 14.678023305410388
Mean Percentage Error: -0.0897794355249639
Mean Absolute Percentage Error: 0.25755392701137764
Mean Squared Error: 498.30126751274

//--------------------------------------//
route: 130 
 direction: 1
Mean Absolute Error (train): 8.741284743463178
Mean Absolute Error (test): 8.734902799174371
Mean Percentage Error: -0.09380288403003197
Mean Absolute Percentage Error: 0.24540558409953525
Mean Squared Error: 164.0988931561381
Root Mean Squared Error: 12.8101090220239
R-square value: 0.6170536513287737
route: 130 
 direction: 2
Mean Absolute Error (train): 9.650889087493644
Mean Absolute Error (test): 9.659957990102132
Mean Percentage Error: -0.08951683589318905
Mean Absolute Percentage Error: 0.24697817629963037
Mean Squared Error: 221.7650684252145
Root Mean Squared Error: 14.89177855144289
R-square value: 0.6600369727345514
//--------------------------------------//
route: 15 
 direction: 1
Mean Absolute Error (train): 16.490609943823777
Mean Absolute Error (test): 16.42392657607661
Mean Percentage Error: -0.17304945212535114
Mean Absolute Percentage Error: 0.3640935926558203
Mean Squared Error: 756.6577994870571
R

MemoryError: Unable to allocate 1.89 GiB for an array with shape (173, 1463198) and data type float64

In [19]:
Line_List[14:]

['46A',
 '33',
 '7',
 '39A',
 '49',
 '1',
 '123',
 '41',
 '67X',
 '59',
 '9',
 '40',
 '239',
 '76',
 '84',
 '53',
 '185',
 '151',
 '13',
 '15B',
 '65B',
 '29A',
 '61',
 '140',
 '79A',
 '38A',
 '31',
 '33B',
 '69',
 '44',
 '42',
 '67',
 '184',
 '238',
 '145',
 '17A',
 '32',
 '27A',
 '17',
 '27X',
 '18',
 '122',
 '54A',
 '66',
 '150',
 '56A',
 '37',
 '27',
 '15A',
 '65',
 '11',
 '47',
 '79',
 '83A',
 '63',
 '4',
 '120',
 '41C',
 '70',
 '84A',
 '220',
 '39X',
 '32X',
 '68A',
 '84X',
 '38',
 '102',
 '270',
 '51X',
 '33X',
 '75',
 '26',
 '66A',
 '31A',
 '111',
 '14C',
 '114',
 '76A',
 '44B',
 '161',
 '7A',
 '43',
 '25',
 '104',
 '33A',
 '16C',
 '42D',
 '31B',
 '66X',
 '31D',
 '33D',
 '41B',
 '40B',
 '7D',
 '46E',
 '38D',
 '118',
 '51D',
 '15D',
 '41A',
 '25D',
 '66B',
 '38B',
 '236',
 '7B',
 '41X',
 '69X',
 '68X',
 '25X',
 '40E',
 '70D',
 '116',
 '77X',
 '16D',
 '33E',
 '41D']

In [5]:
for Route in Line_List[14:]:

    df = pd.read_csv("~/data/Cleaned_By_Route_Header/route_{}.csv".format(Route))
    print("//--------------------------------------//")  
    
    if len(df.DIRECTION.unique()) == 1:

        direction = df.DIRECTION.unique()[0]
        print("route:",Route,"\n","direction:", direction)
        final_df = Final_Clean(df, direction)
        Lin_reg(final_df)  
    
    elif len(df.DIRECTION.unique()) == 2:
        for direction in range(1,3):
            print("route:",Route,"\n","direction:", direction)
            final_df = Final_Clean(df, direction)
            Lin_reg(final_df) 

//--------------------------------------//
route: 46A 
 direction: 1
Mean Absolute Error (train): 16.771498405236912
Mean Absolute Error (test): 16.791428610080843
Mean Percentage Error: -0.1354841175697887
Mean Absolute Percentage Error: 0.33346691201621614
Mean Squared Error: 587.8920070549336
Root Mean Squared Error: 24.246484426715014
R-square value: 0.5967116514769211
route: 46A 
 direction: 2
Mean Absolute Error (train): 16.20599633924401
Mean Absolute Error (test): 832259.9865620832
Mean Percentage Error: 3650.048017369717
Mean Absolute Percentage Error: 3650.5443552352617
Mean Squared Error: 4.3679167902496915e+17
Root Mean Squared Error: 660902170.5403676
R-square value: -279795307406519.0
//--------------------------------------//
route: 33 
 direction: 1
Mean Absolute Error (train): 11.7255966839082
Mean Absolute Error (test): 11.659856728281042
Mean Percentage Error: -0.09738878946270935
Mean Absolute Percentage Error: 0.2689835839934872
Mean Squared Error: 321.948923558364

//--------------------------------------//
route: 239 
 direction: 1
Mean Absolute Error (train): 15.616796033744645
Mean Absolute Error (test): 15.407416158769266
Mean Percentage Error: -0.11624727138823285
Mean Absolute Percentage Error: 0.3355102066214717
Mean Squared Error: 621.8885197006842
Root Mean Squared Error: 24.937692750146
R-square value: 0.8442908253936836
route: 239 
 direction: 2
Mean Absolute Error (train): 24.765247463638847
Mean Absolute Error (test): 24.879787785323682
Mean Percentage Error: -0.35692725413716897
Mean Absolute Percentage Error: 0.6277698569241179
Mean Squared Error: 1848.94236654487
Root Mean Squared Error: 42.999329838322716
R-square value: 0.7474193077388469
//--------------------------------------//
route: 76 
 direction: 1
Mean Absolute Error (train): 17.88187663362787
Mean Absolute Error (test): 17.892757499359256
Mean Percentage Error: -0.08555075822978535
Mean Absolute Percentage Error: 0.30223538213927675
Mean Squared Error: 781.6006786591054

//--------------------------------------//
route: 79A 
 direction: 1
Mean Absolute Error (train): 11.42706115471087
Mean Absolute Error (test): 11.452956617171843
Mean Percentage Error: -0.08185592336688384
Mean Absolute Percentage Error: 0.23828577402671422
Mean Squared Error: 297.6953220316314
Root Mean Squared Error: 17.25384948443771
R-square value: 0.6841190735065494
route: 79A 
 direction: 2
Mean Absolute Error (train): 15.42200696584146
Mean Absolute Error (test): 15.31623141110074
Mean Percentage Error: -0.07693823580868314
Mean Absolute Percentage Error: 0.251024318540442
Mean Squared Error: 642.4963347572391
Root Mean Squared Error: 25.34751141152202
R-square value: 0.7544731744379404
//--------------------------------------//
route: 38A 
 direction: 1
Mean Absolute Error (train): 18.370330777064687
Mean Absolute Error (test): 18.280978453375567
Mean Percentage Error: -0.11114539222248782
Mean Absolute Percentage Error: 0.3298552013190794
Mean Squared Error: 886.492154429045


//--------------------------------------//
route: 32 
 direction: 1
Mean Absolute Error (train): 9.76098228344765
Mean Absolute Error (test): 9.795947953882637
Mean Percentage Error: -0.09485355564986221
Mean Absolute Percentage Error: 0.24836287525557035
Mean Squared Error: 220.95836221861776
Root Mean Squared Error: 14.864668251212933
R-square value: 0.5864745911979803
route: 32 
 direction: 2
Mean Absolute Error (train): 8.819042500945045
Mean Absolute Error (test): 8.846550763950676
Mean Percentage Error: -0.08728664620000681
Mean Absolute Percentage Error: 0.24306239860657372
Mean Squared Error: 181.37831894989762
Root Mean Squared Error: 13.467676820814257
R-square value: 0.6323379704558036
//--------------------------------------//
route: 27A 
 direction: 1
Mean Absolute Error (train): 9.249011160830474
Mean Absolute Error (test): 9.258634640347234
Mean Percentage Error: -0.09815994415809116
Mean Absolute Percentage Error: 0.25502270864475546
Mean Squared Error: 181.170310306151

//--------------------------------------//
route: 15A 
 direction: 1
Mean Absolute Error (train): 15.21971860454452
Mean Absolute Error (test): 15.271719088077118
Mean Percentage Error: -0.14440744359447033
Mean Absolute Percentage Error: 0.3424815464540756
Mean Squared Error: 529.7800075563309
Root Mean Squared Error: 23.016950439976426
R-square value: 0.6384757252366678
route: 15A 
 direction: 2
Mean Absolute Error (train): 17.546586577991807
Mean Absolute Error (test): 17.50999113440094
Mean Percentage Error: -0.16617611795892975
Mean Absolute Percentage Error: 0.3720208440334188
Mean Squared Error: 720.5038522706301
Root Mean Squared Error: 26.842202820756537
R-square value: 0.6624405775792478
//--------------------------------------//
route: 65 
 direction: 1
Mean Absolute Error (train): 11.76828410379508
Mean Absolute Error (test): 11.854655136819375
Mean Percentage Error: -0.08310243540452354
Mean Absolute Percentage Error: 0.26934358064010244
Mean Squared Error: 377.41725368292

//--------------------------------------//
route: 220 
 direction: 1
Mean Absolute Error (train): 11.22205774373644
Mean Absolute Error (test): 11.202939367749861
Mean Percentage Error: -0.08969834710212415
Mean Absolute Percentage Error: 0.23927029519616472
Mean Squared Error: 371.206356929827
Root Mean Squared Error: 19.266716298576334
R-square value: 0.7416428708583145
route: 220 
 direction: 2
Mean Absolute Error (train): 10.282921715510518
Mean Absolute Error (test): 10.275611634874503
Mean Percentage Error: -0.08460480721814095
Mean Absolute Percentage Error: 0.22884883426170455
Mean Squared Error: 288.8908150114002
Root Mean Squared Error: 16.996788373436914
R-square value: 0.7557329495417995
//--------------------------------------//
route: 39X 
 direction: 1
Mean Absolute Error (train): 39.564199323586315
Mean Absolute Error (test): 39.69180273409058
Mean Percentage Error: -0.09333829652650137
Mean Absolute Percentage Error: 0.2713809335252103
Mean Squared Error: 3839.41443383

Mean Absolute Error (train): 13.901558929080982
Mean Absolute Error (test): 13.913979709569
Mean Percentage Error: -0.09584082818711583
Mean Absolute Percentage Error: 0.2768488159811128
Mean Squared Error: 464.1962484056824
Root Mean Squared Error: 21.545214048732085
R-square value: 0.744570918531694
route: 66A 
 direction: 2
Mean Absolute Error (train): 11.870480959643078
Mean Absolute Error (test): 11.954438037628904
Mean Percentage Error: -0.08795759499543379
Mean Absolute Percentage Error: 0.2575825760154383
Mean Squared Error: 362.75589992565654
Root Mean Squared Error: 19.046151840349708
R-square value: 0.708039209937338
//--------------------------------------//
route: 31A 
 direction: 1
Mean Absolute Error (train): 9.4211692142414
Mean Absolute Error (test): 9.397621125394346
Mean Percentage Error: -0.09637009460929402
Mean Absolute Percentage Error: 0.2465825892234722
Mean Squared Error: 249.79694345047713
Root Mean Squared Error: 15.804965784539906
R-square value: 0.63661350

//--------------------------------------//
route: 33A 
 direction: 1
Mean Absolute Error (train): 7.687243701570405
Mean Absolute Error (test): 7.5756881522272606
Mean Percentage Error: -0.06098849921620388
Mean Absolute Percentage Error: 0.18017623103447053
Mean Squared Error: 171.92365131234342
Root Mean Squared Error: 13.111965959090323
R-square value: 0.806983455865543
route: 33A 
 direction: 2
Mean Absolute Error (train): 7.472315691991691
Mean Absolute Error (test): 7.424861198567237
Mean Percentage Error: -0.06956578681388086
Mean Absolute Percentage Error: 0.1931251935300459
Mean Squared Error: 159.48421371086658
Root Mean Squared Error: 12.628705939678325
R-square value: 0.8172526909397725
//--------------------------------------//
route: 16C 
 direction: 1
Mean Absolute Error (train): 13.234774608579842
Mean Absolute Error (test): 13.560348819125151
Mean Percentage Error: -0.13548983463415273
Mean Absolute Percentage Error: 0.32226969591118204
Mean Squared Error: 368.42256064

Mean Absolute Error (train): 20.571634570827385
Mean Absolute Error (test): 20.65898915397611
Mean Percentage Error: -0.1483943181359817
Mean Absolute Percentage Error: 0.33734253039524204
Mean Squared Error: 958.6210469012684
Root Mean Squared Error: 30.961606012952046
R-square value: 0.6623886387323706
//--------------------------------------//
route: 51D 
 direction: 1
Mean Absolute Error (train): 12.599588792604798
Mean Absolute Error (test): 12.563326737859509
Mean Percentage Error: -0.07065869341709448
Mean Absolute Percentage Error: 0.21210730711631973
Mean Squared Error: 425.0866104916206
Root Mean Squared Error: 20.617628634050536
R-square value: 0.8531855874336919
route: 51D 
 direction: 2
Mean Absolute Error (train): 20.155871259307013
Mean Absolute Error (test): 3722.1399304878946
Mean Percentage Error: -53.014559942462014
Mean Absolute Percentage Error: 53.188684747903466
Mean Squared Error: 137094606591.351
Root Mean Squared Error: 370262.8884878297
R-square value: -21671

Mean Absolute Error (train): 7.775258941256397
Mean Absolute Error (test): 7.910120374774769
Mean Percentage Error: -0.06973169617186414
Mean Absolute Percentage Error: 0.2113282147779722
Mean Squared Error: 189.75989613330663
Root Mean Squared Error: 13.775336516154756
R-square value: 0.8887894597789245
//--------------------------------------//
route: 70D 
 direction: 1
Mean Absolute Error (train): 23.962290898227877
Mean Absolute Error (test): 23.94406643446962
Mean Percentage Error: -0.1277542055491921
Mean Absolute Percentage Error: 0.32303456975205735
Mean Squared Error: 1562.8035213908577
Root Mean Squared Error: 39.53230984132925
R-square value: 0.6935496248646084
route: 70D 
 direction: 2
Mean Absolute Error (train): 15.710095923014595
Mean Absolute Error (test): 15.965584933612758
Mean Percentage Error: -0.09372627156868052
Mean Absolute Percentage Error: 0.2570282542810461
Mean Squared Error: 718.589084657046
Root Mean Squared Error: 26.80651198229725
R-square value: 0.85963